
# Random Forest - Classificação


Vamos implementar um modelo de classificação Random Forest utilizando a base de leads da nossa loja automobilistica, a mesma que usamos na aula de regressão logistica. Vamos explorar como o Random Forest pode melhorar a precisão das previsões e a robustez do modelo. Além disso, veremos como ajustar hiperparâmetros para otimizar o desempenho do modelo. Essa prática reforçará a teoria aprendida e demonstrará a aplicação prática desse algoritmo poderoso.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
base = pd.read_csv("CARRO_CLIENTES.csv", delimiter=',')

In [ ]:
# Substituindo os valores da coluna 'GENDER' por números específicos utilizando o replace
gender_mapping = {'Male': 0, 'Female': 1}
base['Gender'] = base['Gender'].replace(gender_mapping)

C:\Users\Úrsula\AppData\Local\Temp\ipykernel_6288\1559211540.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  base['Gender'] = base['Gender'].replace(gender_mapping)


In [ ]:
# Separando em X (variáveis de entrada) e Y (variável de saída)
X = base.drop('Purchased', axis=1)  # X contém todas as colunas exceto 'Purchased'
Y = base['Purchased']  # Y contém apenas a coluna 'Purchased'

In [ ]:
# Separar em base de treino e teste (usando 80% para treino e 20% para teste)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Para essa aula testaremos nosso RF sem fazer a padronização dos dados, como fizemos na regressão logistica.

Random Forest geralmente se adapta melhor a dados despadronizados em comparação com a regressão logística. Isso ocorre porque o Random Forest é baseado em árvores de decisão, que são menos sensíveis à escala dos dados.

Aplicando nosso modelo sem ajustar hyperparametros:


In [ ]:
# Iniciando o modelo Random Forest
rf_model = RandomForestClassifier(random_state=42)

In [ ]:
# Treinando o modelo
rf_model.fit(X_train, Y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Fazendo previsões no conjunto de teste
Y_pred = rf_model.predict(X_test)

In [ ]:
# Avaliando o modelo
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)

print(f"Acurácia: {accuracy:.2f}")
print("Relatório de Classificação:\n", report)
print("Matriz de Confusão:\n", conf_matrix)

Acurácia: 0.91
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.91      0.93      0.92       112
           1       0.91      0.89      0.90        88

    accuracy                           0.91       200
   macro avg       0.91      0.91      0.91       200
weighted avg       0.91      0.91      0.91       200

Matriz de Confusão:
 [[104   8]
 [ 10  78]]


Precisão: Para ambas as classes, 91% das previsões positivas foram corretas.

Recall: Para a classe 0, 93% dos casos reais positivos foram identificados corretamente, enquanto para a classe 1, 89% foram identificados corretamente.

F1: Para a classe 0, o F1-Score é 0.92, e para a classe 1, é 0.90.

Verdadeiros Negativos: 104
Falsos positivos: 8
Falsos Negativos: 10
Verdadeiros Positivos: 78

Nosso objetivo é identificar possíveis compradores de carros a partir dos leads de clientes que entram no site, para enviar eventos promocionais.

**Falsos Positivos:** Clientes que são classificados como potenciais compradores (classe 1), mas que na verdade não comprarão um carro (classe 0).

Impacto: Enviar eventos promocionais para esses clientes que não têm intenção de comprar.
- Custo de envio de promoções e recursos de marketing desnecessários.

**Falsos Negativos:** Clientes que são classificados como não compradores (classe 0), mas que na verdade comprariam um carro (classe 1).

Impacto: Perder oportunidades de conversão porque os clientes que poderiam ser compradores não receberam eventos promocionais.
- Perda de receita potencial devido a oportunidades perdidas.


# Melhorando nosso Modelo.

**Definir o Espaço de Busca dos Hiperparâmetros:** Identificaremos os hiperparâmetros que desejamos otimizar.

Alguns hiperparâmetros importantes para o Random Forest incluem:

n_estimators: Número de árvores na floresta.

max_depth: Profundidade máxima das árvores.

min_samples_split: Número mínimo de amostras necessárias para dividir um nó.

min_samples_leaf: Número mínimo de amostras necessárias para estar em um nó folha.

max_features: Número de recursos a serem considerados para encontrar a melhor divisão.

In [ ]:
# Definir o espaço de busca dos hiperparâmetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

In [ ]:
# Configurando o Randomized Search
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid,
                                   n_iter=100, cv=5, n_jobs=-1, verbose=2, random_state=42, scoring='accuracy')

**estimator=rf_model:**

Este parâmetro especifica o modelo base que será ajustado. No caso, rf_model é o modelo de Random Forest que você deseja otimizar.

**param_distributions=param_grid:**

Este parâmetro contém o dicionário de hiperparâmetros a serem amostrados. param_grid deve ser um dicionário onde as chaves são os nomes dos hiperparâmetros e os valores são listas de valores ou distribuições probabilísticas dos quais serão sorteados.


**n_iter=100:**

Este parâmetro define o número de combinações diferentes de hiperparâmetros a serem testadas. Com 100 iterações, o RandomizedSearchCV vai experimentar 100 combinações aleatórias dos hiperparâmetros especificados.


**cv=5:**
Este parâmetro define o número de dobras para a validação cruzada. cv=5 significa que os dados serão divididos em 5 partes, e o modelo será treinado e validado 5 vezes.



In [ ]:
# Executando o Randomized Search
random_search.fit(X_train, Y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': [None, 10, 20, 30],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200]},
                   random_state=42, scoring='accuracy', verbose=2)

In [ ]:
# Obtendo os melhores hiperparâmetros através do método randomico
best_params = random_search.best_params_
print(f"Melhores Hiperparâmetros: {best_params}")

Melhores Hiperparâmetros: {'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 10}


**n_estimators -  50**

Descrição: Este parâmetro define o número de árvores na floresta.


**min_samples_split- 10**

Descrição: Este parâmetro define o número mínimo de amostras necessárias para dividir um nó.

**min_samples_leaf- 1**

Descrição: Este parâmetro especifica o número mínimo de amostras que um nó folha deve ter.


**max_features - 'log2'**

Descrição: Este parâmetro define o número de características a serem consideradas ao procurar a melhor divisão. Com log2, o modelo considera o logaritmo de base 2 do número total de características.

**max_depth: - 10**

Descrição: Este parâmetro especifica a profundidade máxima das árvores na floresta.

In [ ]:
# Treinando o modelo com os melhores hiperparâmetros encontrados acima
best_rf_model = random_search.best_estimator_
best_rf_model.fit(X_train, Y_train)

RandomForestClassifier(max_depth=10, max_features='log2', min_samples_split=10,
                       n_estimators=50, random_state=42)

In [ ]:
Y_pred = best_rf_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)

print(f"Acurácia: {accuracy:.2f}")
print("Relatório de Classificação:\n", report)
print("Matriz de Confusão:\n", conf_matrix)

Acurácia: 0.91
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.91      0.94      0.92       112
           1       0.92      0.88      0.90        88

    accuracy                           0.91       200
   macro avg       0.91      0.91      0.91       200
weighted avg       0.91      0.91      0.91       200

Matriz de Confusão:
 [[105   7]
 [ 11  77]]



# Random Forest - Regressão


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

Vamos abordar um problema de previsão de preços de aluguel. O objetivo é prever o valor do aluguel de imóveis com base em diversas características como localização, número de quartos etc.

Random Forest é uma técnica versátil de aprendizado de máquina que pode ser utilizada tanto para problemas de regressão quanto para classificação assim como as árvores de decisão.

No caso da regressão, o Random Forest faz a previsão tomando a média das previsões de todas as árvores da floresta. Cada árvore individual faz uma estimativa para o valor contínuo do alvo, e o Random Forest combina essas estimativas, calculando a média para produzir a previsão final. Isso ajuda a suavizar as previsões e a reduzir a variabilidade, resultando em um modelo mais estável e preciso.

In [ ]:
base_imov = pd.read_csv("ALUGUEL_MOD12.csv", delimiter=';')

In [ ]:
base_imov.describe()

,Valor_Aluguel,Valor_Condominio,Metragem,N_Quartos,N_banheiros,N_Suites,N_Vagas
count,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.000000,7203.00000
mean,2966.596140,811.538109,88.506178,2.300153,2.095932,1.016660,1.44176
std,2948.720385,796.564846,61.567505,0.826615,0.983812,0.874204,0.86993
min,480.000000,0.000000,30.000000,1.000000,1.000000,0.000000,0.00000
25%,1350.000000,395.000000,52.000000,2.000000,2.000000,1.000000,1.00000
50%,2000.000000,592.000000,67.000000,2.000000,2.000000,1.000000,1.00000
75%,3200.000000,980.000000,100.000000,3.000000,2.000000,1.000000,2.00000
max,25000.000000,9500.000000,880.000000,10.000000,8.000000,5.000000,9.00000


In [ ]:
X = base_imov.drop('Valor_Aluguel', axis=1)
y = base_imov['Valor_Aluguel']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# Notem que usamos o random forest regressor
rf_model_reg = RandomForestRegressor(random_state=42)

In [ ]:
rf_model_reg.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
y_pred_reg = rf_model_reg.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred_reg)
mae = mean_absolute_error(y_test, y_pred_reg)
r2 = r2_score(y_test, y_pred_reg)

In [ ]:
print(f"RMSE: {mse ** 0.5}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

RMSE: 1872.2883535577132
MAE: 968.6055994375845
R²: 0.6434247719501688


O RMSE é a raiz quadrada da média dos erros quadráticos.
Um RMSE de 1872.29 significa que, em média, a diferença entre os valores previstos e os valores reais é de aproximadamente 1872.29 unidades (provavelmente na mesma unidade que os preços de aluguel).

O MAE é a média dos erros absolutos.
Um MAE de 968.61 significa que, em média, a diferença entre os valores previstos e os valores reais é de aproximadamente 968.61 unidades.

O R² mede a proporção da variância nos dados de resposta que é explicada pelo modelo. Um R² de 0.6434 indica que aproximadamente 64.34% da variância nos preços de aluguel pode ser explicada pelas variáveis independentes (features) no modelo.

**Vamos tentar melhorar os hyperparametros:**

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
rf_model_reg_best = RandomForestRegressor(random_state=42)

In [ ]:
# Definindo o RandomizedSearchCV para regressão:
random_search_reg = RandomizedSearchCV(estimator=rf_model_reg_best, param_distributions=param_grid,
                                   n_iter=50, cv=5, n_jobs=-1, verbose=2, random_state=42, scoring='neg_mean_squared_error')

In [ ]:
random_search_reg.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_depth': [None, 10, 20, 30],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [ ]:
best_params_reg = random_search_reg.best_params_
print(f"Melhores Hiperparâmetros: {best_params_reg}")

Melhores Hiperparâmetros: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10}


In [ ]:
best_rf_model_rg = random_search_reg.best_estimator_
best_rf_model_rg.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, max_features='sqrt', random_state=42)

In [ ]:
y_pred_reg2 = best_rf_model_rg.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred_reg2)
mae = mean_absolute_error(y_test, y_pred_reg2)
r2 = r2_score(y_test, y_pred_reg2)

print(f"Melhores hiperparâmetros: {random_search.best_params_}")
print(f"RMSE: {mse ** 0.5}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

Melhores hiperparâmetros: {'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 10}
RMSE: 1858.0023460338516
MAE: 975.1161060374673
R²: 0.6488455203339903
